In [1]:
import os
import shutil

import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras

from keras import layers
from keras import models
from keras import optimizers

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

2023-11-20 18:12:31.646677: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 18:12:31.688875: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 18:12:31.688908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 18:12:31.689969: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 18:12:31.696541: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 18:12:31.696978: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
import tensorflow as tf


In [3]:
model = tf.keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

2023-11-20 18:14:03.779476: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% of free system memory.
2023-11-20 18:14:03.839662: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% of free system memory.
2023-11-20 18:14:03.863393: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% of free system memory.


In [4]:
model.compile(loss='binary_crossentropy',optimizer=optimizers.SGD(learning_rate=.002,momentum=.8),metrics=['acc'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11215873 (42.79 MB)
Trainable params: 11215873 (42.79 MB)
Non-trainable params: 0 (0.00 Byte)
______________

## Question 1


#### Best loss function : binary crossentropy

## Question 2

#### Total number of parameters in the convolutional layer : 896

In [8]:
train_dir = 'data/train'
test_dir = 'data/test'

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_gen = train_datagen.flow_from_directory(train_dir, target_size=(150,150), batch_size = 20, class_mode = 'binary', shuffle=True)

Found 3677 images belonging to 2 classes.


In [11]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
val_gen = val_datagen.flow_from_directory(test_dir, target_size=(150,150), batch_size = 20, class_mode = 'binary', shuffle=True)

Found 918 images belonging to 2 classes.


## Question 3 

In [14]:
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen
)

Epoch 1/10


2023-11-20 18:25:46.150838: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% of free system memory.
2023-11-20 18:25:46.459044: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% of free system memory.


184/184 [==============================] - 74s 398ms/step - loss: 0.6837 - acc: 0.5532 - val_loss: 0.6458 - val_acc: 0.6046
Epoch 2/10
184/184 [==============================] - 72s 392ms/step - loss: 0.6581 - acc: 0.5869 - val_loss: 0.6385 - val_acc: 0.5697
Epoch 3/10
184/184 [==============================] - 76s 415ms/step - loss: 0.6443 - acc: 0.6160 - val_loss: 0.6299 - val_acc: 0.5871
Epoch 4/10
184/184 [==============================] - 71s 387ms/step - loss: 0.6287 - acc: 0.6345 - val_loss: 0.5900 - val_acc: 0.6721
Epoch 5/10
184/184 [==============================] - 72s 391ms/step - loss: 0.5969 - acc: 0.6845 - val_loss: 0.5833 - val_acc: 0.6841
Epoch 6/10
184/184 [==============================] - 77s 416ms/step - loss: 0.5541 - acc: 0.7199 - val_loss: 0.5458 - val_acc: 0.7048
Epoch 7/10
184/184 [==============================] - 72s 392ms/step - loss: 0.5223 - acc: 0.7571 - val_loss: 0.5513 - val_acc: 0.7266
Epoch 8/10
184/184 [==============================] - 72s 391ms/st

In [22]:
acc = history.history['acc']
print("median accuracy:", round(np.median(acc), 3))

median accuracy: 0.702


In [23]:
loss = history.history['loss']
print("standard deviation of training loss:", np.std(history.history['loss']))

standard deviation of training loss: 0.08166098691605865


## Data augmentation

In [24]:
image_gen = ImageDataGenerator(rescale=1./255,
                               rotation_range=50,
                               width_shift_range=0.1,
                                height_shift_range=0.1,
                                zoom_range=0.1,
                                horizontal_flip=True,
                                fill_mode='nearest')

In [25]:
train_data = image_gen.flow_from_directory(
    './data/train',
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

Found 3677 images belonging to 2 classes.


In [26]:
val_data = image_gen.flow_from_directory(
    './data/test',
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

Found 918 images belonging to 2 classes.


## Question 5

In [27]:
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

Epoch 1/10
184/184 [==============================] - 87s 475ms/step - loss: 0.5072 - acc: 0.7612 - val_loss: 0.5029 - val_acc: 0.7821
Epoch 2/10
184/184 [==============================] - 88s 477ms/step - loss: 0.4965 - acc: 0.7716 - val_loss: 0.4886 - val_acc: 0.7778
Epoch 3/10
184/184 [==============================] - 87s 475ms/step - loss: 0.4857 - acc: 0.7789 - val_loss: 0.4840 - val_acc: 0.7669
Epoch 4/10
184/184 [==============================] - 87s 471ms/step - loss: 0.4841 - acc: 0.7756 - val_loss: 0.5482 - val_acc: 0.7124
Epoch 5/10
184/184 [==============================] - 86s 468ms/step - loss: 0.4932 - acc: 0.7716 - val_loss: 0.5060 - val_acc: 0.7473
Epoch 6/10
184/184 [==============================] - 88s 476ms/step - loss: 0.4727 - acc: 0.7852 - val_loss: 0.5088 - val_acc: 0.7538
Epoch 7/10
184/184 [==============================] - 88s 475ms/step - loss: 0.4697 - acc: 0.7862 - val_loss: 0.4761 - val_acc: 0.7756
Epoch 8/10
184/184 [==============================] - 9

In [28]:
np.mean(history.history['val_loss'])

0.49332896769046786

## Question 6

In [30]:
np.average(history.history['val_acc'][5:])

0.7771241784095764